## Notebook for scan the public Jerber et al iPSC derived DA neruon cell-type eQTL results

- [Jerber et. al.](https://pubmed.ncbi.nlm.nih.gov/33664506/)
    - Jerber J, Seaton DD, Cuomo ASE et al. Population-scale single-cell RNA-seq profiling across dopaminergic neuron differentiation. Nat Genet 2021;53:304–12.
    - 175 individuals for single-cell, 193 for psuedobulk
    - using their day 52 untreated DA neuron results
    
![HipSci iPSC DAn cartoon](https://media.springernature.com/m312/springer-static/image/art%3A10.1038%2Fs41588-021-00801-6/MediaObjects/41588_2021_801_Fig1_HTML.png?as=webp) 
    
Scan the Jerber results for any nominal intersect with PD risk and for these intersects then run colocalization, results have already have additional prep for use in the meta-eQTL analysis with FOUNDIN-PD iPSC derived DA neuron cell-type eQTL

In [1]:
!date

Mon Apr 10 17:24:30 UTC 2023


#### import libraries

In [2]:
from pandas import read_csv, read_parquet, DataFrame, Series, merge, concat
from threading import Thread
from numpy import around
import colocalization as clc
import concurrent.futures

#### set notebook variables

In [3]:
# naming
cohort = 'hipsci'
day = 'D52'
set_name_frmt = 'foundin_{day}_{cohort}-{cell_type}'
dx = 'PD'

# directories 
wrk_dir = '/labshare/raph/datasets/foundin_qtl'
public_dir = f'{wrk_dir}/public'
results_dir = f'{wrk_dir}/results'

# input files
# with agreement in place use full summary stats
# gwas_sum_stats_file = f'{public_dir}/nalls_pd_gwas/pd_sumstats_no23andme_buildGRCh38.tsv.gz'
gwas_sum_stats_file = f'{public_dir}/nalls_pd_gwas/pd_sumstats_23andme_buildGRCh38.tsv.gz'
index_variants_file = f'{public_dir}/nalls_pd_gwas/index_variants.list'  

# variables
DEBUG = False
max_nominal = 0.01
cell_types = ['DA', 'pseudobulk']
min_h4 = 0.5

#### functions

In [4]:
def load_eqtl_results(in_file: str, variants: list, verbose: bool=False):
    # have to do pass to find all features to possible capture
    qtl_df = read_parquet(in_file)
    if verbose:
        print(f'read {qtl_df.shape}')
    qtl_df.rename(columns={'id': 'variant_id'}, inplace=True)
    # find traits tested against variants of interest with sufficicent p-value
    oi_results = qtl_df.loc[(qtl_df['variant_id'].isin(variants)) & 
                            (qtl_df['p_value'] <= max_nominal)]
    phenos_oi = list(oi_results['feature_id'].unique())
    # do pass to keep results that belong those phenos
    possible_results_oi = qtl_df.loc[qtl_df['feature_id'].isin(phenos_oi)].copy()
    if verbose:
        display(possible_results_oi.head())
        print(phenos_oi)
    return phenos_oi, possible_results_oi

def process_qtl(trait_df: DataFrame, other_stats: DataFrame) -> DataFrame:
    """ Prep the QTL (or trait2) results for use in colocalization. 
        Where prep performs Wakefield Approx Bayes Factor, posterior probabliltiy,
        and credible sets calculations and identification
    Args:
        trait_df (pandas.DataFrame) QTL results for a feature
        other_stats (pandas.DataFrame) trait1 (or risk) stats to be used with these
            qtl (or trait2) stats for colocalization
    Returns:
        (pandas.DataFrame) qtl results with ABF, PP, and credible sets computed
    """
    # some feature QTL stats may also have multiple results per variants 
    # so need to reduce or remove these
    # these are typically a results of variants that are multi-allelic like indels
    trait_df = trait_df.drop_duplicates(subset=['variant_id'], keep='first')
    # calculate the ABF's for the feature's QTL results
    trait_df["logABF"] = trait_df.apply(
    lambda result: clc.calc_abf(pval=result.p_value, maf=clc.freq_to_maf(result.maf),
                                n=result.n_samples), 
                                axis=1)    
    trait_df = trait_df.sort_values("logABF", ascending=False)
    # calculate the posterior probability for each variant
    trait_df['PP'] = clc.compute_pp(trait_df.logABF)
    # identify the credible set(s), 95% and 99%, the the posterior probabilities
    clc.credible_sets(trait_df)
    # subset the feature QTL variants to just those present in the GWAS
    trait_df = trait_df.loc[trait_df.variant_id.isin(other_stats.variant_id)] 
    return trait_df

def process_gwas(trait_stats: DataFrame, other_stats: DataFrame) -> DataFrame:
    """ Prep the risk (or trait1) results for use in colocalization. 
        Where prep performs subet of variant to those present in other (trait2/qtl),
        Wakefield Approx Bayes Factor, posterior probabliltiy,
        and credible sets calculations and identification
    Args:
        trait_stats (pandas.DataFrame) trait1 (or risk) stats to be used with these
            qtl (or trait2) stats for colocalization
        other_stats (pandas.DataFrame) trait2 (or qtl) stats to be used with these
            risk (or trait2) stats for colocalization
    Returns:
        (pandas.DataFrame) risk results with ABF, PP, and credible sets computed
    """ 
    # subset the risk summary stats by the feature's QTL variants present
    ret_df = trait_stats.loc[trait_stats.variant_id.isin(other_stats.variant_id)].copy()
    # calculate the ABF's for the risk results
    ret_df['logABF'] = ret_df.apply(
        lambda result: clc.calc_abf(pval=result.p_value, 
                                    maf=clc.freq_to_maf(result.effect_allele_frequency),
                                    n=result.n_total, 
                                    prop_cases=result.case_prop), axis=1)
    ret_df = ret_df.sort_values('logABF', ascending=False)  
    # calculate the posterior probability for each variant
    ret_df['PP'] = clc.compute_pp(ret_df.logABF)
    # identify the credible set(s), 95% and 99%, the the posterior probabilities
    clc.credible_sets(ret_df)
    return ret_df

def ensure_matched_indices(df1: DataFrame, df2: DataFrame) -> {DataFrame, DataFrame}:
    """ make sure the two datasets are ordered the same
        modifies both df1 and df2
    Args:
        df1 (pandas.DataFrame) risk or trait1 data
        df2 (pandas.DataFrame) qtl or trait2 data
    """ 
    # ensure that the risk and feature variants ABF's are ordered the same
    df1.set_index('variant_id', inplace=True)
    df2.set_index('variant_id', inplace=True)
    shared_indices = df1.index.intersection(df2.index)
    df1 = df1.loc[shared_indices,]
    df2 = df2.loc[shared_indices,]
    temp = df1.index.values == df2.index.values
    return df1, df2

def colocalize(t1_abfs, t2_abfs, feature: str) -> Series:
    """ Perform the colocalization between trait1 and trait2 ABFs
    Args:
        t1_abfs (array_like) trait1's ABFs
        t2_abfs (array_like) trait2's ABFs
        feature (string) trait2's name or ID
    Returns:
        (pandas.Series) named colocalization posterior probabilities
    """
    h_probs = clc.combine_abf(t1_abfs, t2_abfs)
    names = [f'H{x}' for x in range(5)]
    cl_result = Series(data=around(h_probs, decimals=3), index=names)
    cl_result['feature'] = feature
    return cl_result  

def compute_combined_pp(t1_df: DataFrame, t2_df: DataFrame) -> DataFrame:
    """ Compute the the combined ABFs posterior probabilities and credible sets
    Args:
        t1_df (pandas.DataFrame) risk or trait1's data
        t2_df (pandas.DataFrame) qtl or trait2's data
    Returns:
        (pandas.DataFrame) t1_df combined with t2_df with PP and credible sets ID'd
    """
    ret_df = merge(t1_df, t2_df, how='inner', on='variant_id', suffixes=('_risk', '_qtl'))
    # calculate the posterior probability for each variant
    ret_df['PP'] = clc.compute_pp(ret_df.logABF_risk + ret_df.logABF_qtl)
    # identify the credible set(s), 95% and 99%, the the posterior probabilities
    clc.credible_sets(ret_df)
    ret_df.rename(columns={'PP': 'h4_pp'}, inplace=True)
    return ret_df

def process_results(cell_type: str, verbose: bool=False):
    coloc_scores = []
    coloc_h4_pps = None 
    this_file = f'{public_dir}/jerber_da_neuron_eqtl/{cohort}_{day}_{cell_type}.untreated.qtl_results_all.hg38.parquet'
    features_oi, results_to_test = load_eqtl_results(this_file, index_variants)
    if verbose:
        print(features_oi)
        print(results_to_test.shape)
    for feature in features_oi:
        feature_df = results_to_test.loc[results_to_test.feature_id == feature]
        feature_df = process_qtl(feature_df, gwas_stats_df)
        # prep the GWAS results
        risk_df = process_gwas(gwas_stats_df, feature_df)
        # ensure that the risk and feature variants ABF's are ordered the same
        risk_df, feature_df = ensure_matched_indices(risk_df, feature_df)
        # perform the colocalization
        cl_result = colocalize(risk_df.logABF, feature_df.logABF, feature)
        # if H4 is supported then compute H4.PP the H4 credible sets
        cl_result['h4_supported'] = clc.h4_supported(cl_result)
        if cl_result.H4 >= min_h4:
            combined_df = compute_combined_pp(risk_df, feature_df)
            coloc_h4_pps = concat([coloc_h4_pps, combined_df])                  
        # add these scores to the rest
        coloc_scores.append(cl_result)
    # create a dataframe from the list of coloc scores    
    coloc_scores_df = DataFrame(coloc_scores)
    ### save the result files for this cell-type
    set_name = set_name_frmt.format(day=day, cohort=cohort, cell_type=cell_type)    
    coloc_scores_files = f'{results_dir}/{set_name}_{dx}.coloc.pp.csv'
    coloc_casuals_files = f'{results_dir}/{set_name}_{dx}.casuals.pp.parquet'
    if verbose:
        print(set_name, coloc_scores_files)
    coloc_scores_df.to_csv(coloc_scores_files, index=False)
    if not coloc_h4_pps is None:
        coloc_h4_pps.to_parquet(coloc_casuals_files)
    else:
        print(f'{cell_type} no H4 supported so no coloc_h4_pps')    

### load the input data

#### load the risk variants of interest

In [5]:
%%time
variants_oi_df = read_csv(index_variants_file)
print(variants_oi_df.shape)
index_variants = list(variants_oi_df.variant.unique())
if DEBUG:
    display(variants_oi_df.head())
    print(index_variants)

(91, 1)
CPU times: user 1.32 ms, sys: 2.25 ms, total: 3.57 ms
Wall time: 3.15 ms


#### load the full gwas summary stats

In [6]:
%%time
gwas_stats_df = read_csv(gwas_sum_stats_file, sep='\t')
print(gwas_stats_df.shape)
if DEBUG:
    display(gwas_stats_df.sample(5))

(7769022, 12)
CPU times: user 9.84 s, sys: 913 ms, total: 10.8 s
Wall time: 10.8 s


#### set case proportion for GWAS summary stats

In [7]:
gwas_stats_df['n_total'] = gwas_stats_df.n_cases + gwas_stats_df.n_controls
    
gwas_stats_df['case_prop'] = gwas_stats_df.n_cases / gwas_stats_df.n_total

#### subset index variant stats

In [8]:
index_stats_df = gwas_stats_df.loc[gwas_stats_df.variant_id.isin(index_variants)]
print(index_stats_df.shape)
if DEBUG:
    display(index_stats_df.head())

(88, 13)


### load data, format data, and save new input files

#### Jerber et al result

#### load and analyze the Jerber results

In [9]:
%%time
fs_list = []
with concurrent.futures.ProcessPoolExecutor() as ppe:
    for cell_type in cell_types:
        print(f'#### {cell_type} ####')
        fs_list.append(ppe.submit(process_results, cell_type, DEBUG))
for future in concurrent.futures.as_completed(fs_list):
    print('a cell-type finished')        

#### DA ####
#### pseudobulk ####
a cell-type finished
a cell-type finished
CPU times: user 53.8 ms, sys: 530 ms, total: 584 ms
Wall time: 3min 15s


In [10]:
!date

Mon Apr 10 17:27:58 UTC 2023
